In [177]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

Todo:
1. Dropdown menu
2. Borough bar chart (req 1)
3. Neighborhood bar chart (req 1,2)
4. Map of NYC (req 1, 2*/3*)
5. Scatterplots (req 1, 2*/3*)

In [178]:
# load in the data csv files
tree_df = pd.read_csv('./final_data/trees_sample.csv')
borough_df = pd.read_csv('./final_data/boroughs_sample.csv')
neighborhood_df = pd.read_csv('./final_data/neighborhoods_sample.csv')

In [179]:
# make the borough graph
fig = px.bar(y=borough_df["Borough"], x=borough_df["Number of Trees"], orientation='h')
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
borough_bar = go.Figure(data=fig)

In [180]:
# make the neighborhood graph
fig = px.bar(x=neighborhood_df["UHF34 Name"], y=neighborhood_df["Number of Trees"], orientation='h')
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
neighborhood_bar = go.Figure(data=fig)

In [181]:
# TODO: make the map ???
fig = go.Figure(data=go.Scattergeo(
        lon = tree_df['Long'],
        lat = tree_df['Lat'],
        text = tree_df['Diameter'],
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'blue'
        )
        ))

fig.update_layout(geo = dict(
        landcolor = "rgb(212, 212, 212)",
        resolution = 50,
        lonaxis = dict(
            showgrid = False,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = False,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    margin=dict(l=20, r=20, t=20, b=20)
    )
map = go.Figure(data=fig)

In [182]:
# make the scatterplots
fig1 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Fine Particle Pollution"])
fig2 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Diameter"])
fig3 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Summer Surface Temperature"])
fig1.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig2.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig3.update_layout(margin=dict(l=20, r=20, t=20, b=20))
metric1_scatter = go.Figure(data=fig1)
metric2_scatter = go.Figure(data=fig2)
metric3_scatter = go.Figure(data=fig3)

In [ ]:
app = Dash(__name__) # TODO: add stylesheet
app.layout = html.Div([
    html.Div([
        dcc.Dropdown(
            ['Number of Trees', 'Average Status', 'Average Diameter', 'Average Fine Particle Pollution', 'Average Ozone Pollution', 'Average Summer Surface Temperature'],
            'Number of Trees',
            id='metric-dropdown'
        ),
        html.Div([
            html.Div(
                [dcc.Graph(id='borough-bar', figure=borough_bar),
                    html.Button('Deselect Borough', id='borough-deselect-button')],
                    id='borough-div',
                    className='column'
            ),
            html.Div(
                [ dcc.Graph(id='neighborhood-bar', figure=neighborhood_bar),
                html.Button('Deselect Neighborhood', id='neighborhood-deselect-button')],
                id='neighborhood-div',
                className='column'
            )],
            id='bar-div',
            className='row'
        ),
    ],
    id='far-left-div',
    className='column'
    ),
    html.Div([
        dcc.Graph(id='map', figure=map)
        ],
        id='map-div',
        className='column'
    ),
    html.Div([
            dcc.Graph(id='metric1-scatter', className='scatter', figure=metric1_scatter),
            dcc.Graph(id='metric2-scatter', className='scatter', figure=metric2_scatter),
            dcc.Graph(id='metric3-scatter', className='scatter', figure=metric3_scatter)
        ],
        id='scatter-div',
        className='column'
    )
],
className='row'
)

# metric-dropdown to borough-bar: 
# * generate new borough bar graph based on chosen metric
# * sort new borough bar graph by amount of metric per borough
@app.callback(
    Output("borough-bar", "figure"),
    Input("metric-dropdown", "value"),
)
def update_boroughbar_from_metricdropdown(chosen_metric):
    fig = px.bar(
        y=borough_df["Borough"], 
        x=borough_df[chosen_metric], 
        title=f'{chosen_metric} by Borough', 
        orientation='h', 
    )
    fig.update_yaxes(title="Borough")
    fig.update_xaxes(title=chosen_metric)
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=20, r=20, t=20, b=20))
    borough_bar = go.Figure(data=fig)
    return borough_bar
    

# metric-dropdown to neighborhood-bar
# * generate new neighborhood bar graph based on chosen metric
# * sort new neighborhood bar graph by amount of metric per borough
# * must keep in mind if there is a borough selected---then filtering is needed. but if not, do not filter by borough
@app.callback(
    Output("neighborhood-bar", "figure"),
    Input("metric-dropdown", "value"),
)
def update_neighborhoodbar_from_metricdropdown(chosen_metric):
    neighborhood_df_filtered = neighborhood_df
    # TODO: if borough is selected, filter neighborhood_df_filtered by borough
    fig = px.bar(
        y=neighborhood_df_filtered["UHF34 Name"], 
        x=neighborhood_df_filtered[chosen_metric], 
        title=f'{chosen_metric} by Neighborhood', 
        orientation='h', 
    )
    fig.update_yaxes(title="Neighborhood")
    fig.update_xaxes(title=chosen_metric)
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=0, r=0, t=0, b=0))
    borough_bar = go.Figure(data=fig)
    return borough_bar

# metric-dropdown to map
# * switch map coloring based on the chosen metric
# * Keep in mind if we are zoomed into a neighborhood or not (perhaps there's a way to change the map coloring without redrawing the whole map?)

# metric-dropdown to metric1-scatter
# * switch graph axes and data presented based on the chosen metric
# * keep in mind whether we need to highlight a single neighborhood or a borough

# metric-dropdown to metric2-scatter
# * switch graph axes and data presented based on the chosen metric
# * keep in mind whether we need to highlight a single neighborhood or a borough


# metric-dropdown to metric3-scatter
# * switch graph axes and data presented based on the chosen metric
# * keep in mind whether we need to highlight a single neighborhood or a borough

# borough-bar to neighborhood-bar - Liberty
# * deselect neighborhood, if selected
# * filter neighborhoods based on chosen borough

# borough-bar to map
# * zoom into chosen borough

# borough-bar to metric1-scatter
# * highlight all points in the borough

# borough-bar to metric2-scatter
# * highlight all points in the borough

# borough-bar to metric3-scatter
# * highlight all points in the borough

# neighborhood-bar to map
# * zoom into neighborhood
# * show tree locations as points?
# * ability to click on point and have tooltip appear with tree info

# neighborhood-bar to metric1-scatter
# * highlight the singular point representing the neighborhood

# neighborhood-bar to metric2-scatter
# * highlight the singular point representing the neighborhood

# neighborhood-bar to metric3-scatter
# * highlight the singular point representing the neighborhood

# borough-deselect-button to borough-bar - Liberty
# * stop highlighting boroughs on the bar chart

# borough-deslect-button to neighborhood-bar - Liberty
# * stop filtering the neighborhoods
# * stop highlighting neighborhoods, if highlighted

# borough-deselect-button to map
# * zoom out from borough

# borough-deselect-button to metric1-scatter
# * stop highlighting points

# borough-deselect-button to metric2-scatter
# * stop highlighting points

# borough-deselect-button to metric3-scatter
# * stop highlighting points

# neighborhood-deselect-button to map
# * zoom out to borough (or to whole NYC map, if borough is not selected)
# * stop showing individual trees

# neighborhood-deselect-button to metric1-scatter
# * stop highlighting the neighborhood, start highlighting the whole borough (or nothing if no borough selected)

# neighborhood-deselect-button to metric2-scatter
# * stop highlighting the neighborhood, start highlighting the whole borough (or nothing if no borough selected)

# neighborhood-deselect-button to metric3-scatter
# * stop highlighting the neighborhood, start highlighting the whole borough (or nothing if no borough selected)

if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[185], line 64, in update_boroughbar_from_metricdropdown(chosen_metric='Number of Trees')
     62 fig.update_xaxes(title=chosen_metric)
     63 fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=20, r=20, t=20, b=20))
---> 64 borough_bar = go.Figure(data=fig, ticklabelposition="inside")
        fig = Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'h',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([1, 5, 0, 0, 0]),
              'xaxis': 'x',
              'y': array(['Qu